In [1]:
from pathways import Pathways
p = Pathways(datapackage="datapackage_sample/datapackage.json", debug=True)

Invalid datapackage: Descriptor validation error: {'path': 'mapping/mapping.yaml', 'profile': 'data-resource', 'name': 'mapping', 'format': 'yaml', 'mediatype': 'text/yaml', 'encoding': 'utf-8'} is not valid under any of the given schemas at "resources/17" in descriptor and at "properties/resources/items/oneOf" in profile
Invalid datapackage: Descriptor validation error: 'data-resource' is not one of ['tabular-data-resource'] at "resources/17/profile" in descriptor and at "properties/resources/items/properties/profile/enum" in profile
Invalid datapackage: Descriptor validation error: {'path': 'classifications/classifications.yaml', 'profile': 'data-resource', 'name': 'classifications', 'format': 'yaml', 'mediatype': 'text/yaml', 'encoding': 'utf-8'} is not valid under any of the given schemas at "resources/18" in descriptor and at "properties/resources/items/oneOf" in profile
Invalid datapackage: Descriptor validation error: 'data-resource' is not one of ['tabular-data-resource'] at "r

In [2]:
import numpy as np
p.calculate(
    methods=['EF v3.1 EN15804 - climate change - global warming potential (GWP100)',],
    regions=["EU",],
    scenarios=[
        "Scenario A",
    ],
    variables=[
        "technology A",
    ],
    years=[
        2020,
        2030,
        2040,
        2050
    ],
    use_distributions=300,
    multiprocessing=False
)

[(0,  1. , nan, nan,  nan,   nan, False)
 (5,  0.5, nan, nan, 0.4 ,  0.6 , False)
 (5,  0.8, nan, nan, 0.7 ,  0.9 , False)
 (0,  1. , nan, nan,  nan,   nan, False)
 (0,  1. , nan, nan,  nan,   nan, False)
 (5,  0.1, nan, nan, 0.08,  0.2 , False)
 (5,  0.6, nan, nan, 0.5 ,  0.7 , False)
 (0,  1. , nan, nan,  nan,   nan, False)
 (5,  0.1, nan, nan, 0.08,  0.2 , False)
 (5, 10. , nan, nan, 8.  , 12.  , False)
 (5,  0.2, nan, nan, 0.15,  0.25, False)
 (0,  1. , nan, nan,  nan,   nan, False)
 (0,  1. , nan, nan,  nan,   nan, False)]
[(5, 1.5, nan, nan, 1., 1.7, False) (5, 2. , nan, nan, 1., 7. , False)
 (5, 2.3, nan, nan, 1., 2.7, False) (5, 1.2, nan, nan, 1., 1.7, False)
 (5, 1.5, nan, nan, 1., 2. , False)]
Calculating LCA results for some model...
--- Calculating LCA results for Scenario A...
------ Calculating LCA results for 2020...
[(0,  1. , nan, nan,  nan,   nan, False)
 (5,  0.5, nan, nan, 0.4 ,  0.6 , False)
 (5,  0.8, nan, nan, 0.7 ,  0.9 , False)
 (0,  1. , nan, nan,  nan,   nan,

0% [#] 100% | ETA: 00:00:01
Total time elapsed: 00:00:00


------ Calculating LCA results for 2030...
[(0, nan, nan, nan,  nan,  nan, False)
 (5, nan, nan, nan, 0.4 ,  0.6, False)
 (5, nan, nan, nan, 0.4 ,  1.2, False)
 (0, nan, nan, nan,  nan,  nan, False)
 (0, nan, nan, nan,  nan,  nan, False)
 (5, nan, nan, nan, 1.4 ,  1.8, False)
 (5, nan, nan, nan, 0.4 ,  1.2, False)
 (0, nan, nan, nan,  nan,  nan, False)
 (5, nan, nan, nan, 0.04,  0.2, False)
 (5, nan, nan, nan, 4.  , 12. , False)
 (5, nan, nan, nan, 0.1 ,  0.3, False)
 (0, nan, nan, nan,  nan,  nan, False)
 (0, nan, nan, nan,  nan,  nan, False)]
[(5, nan, nan, nan, 1. , 1.7, False) (5, nan, nan, nan, 0.5, 1.7, False)
 (5, nan, nan, nan, 1. , 2.7, False) (5, nan, nan, nan, 1. , 1.7, False)
 (5, nan, nan, nan, 1. , 1.5, False)]


/Users/romain/anaconda3/envs/pathways/lib/python3.11/site-packages/scikits/umfpack/umfpack.py:578: UmfpackWarning: Singular matrix
  warnings.warn('Singular matrix', UmfpackWarning)
/Users/romain/anaconda3/envs/pathways/lib/python3.11/site-packages/scikits/umfpack/umfpack.py:727: UmfpackWarning: Zeroing nan and inf entries...
  warnings.warn('Zeroing nan and inf entries...', UmfpackWarning)
/Users/romain/anaconda3/envs/pathways/lib/python3.11/site-packages/scikits/umfpack/umfpack.py:732: RuntimeWarning: divide by zero encountered in scalar divide
  econd = 1.0 / self.info[UMFPACK_RCOND]
/Users/romain/anaconda3/envs/pathways/lib/python3.11/site-packages/scikits/umfpack/umfpack.py:736: UmfpackWarning: (almost) singular matrix! (estimated cond. number: inf)
  warnings.warn(msg, UmfpackWarning)
0% [#] 100% | ETA: 00:00:01
Total time elapsed: 00:00:00
/Users/romain/anaconda3/envs/pathways/lib/python3.11/site-packages/scikits/umfpack/umfpack.py:578: UmfpackWarning: Singular matrix
  warnings

------ Calculating LCA results for 2040...
[(0, nan, nan, nan,  nan,  nan, False)
 (5, nan, nan, nan, 0.3 ,  0.6, False)
 (5, nan, nan, nan, 0.8 ,  1.2, False)
 (0, nan, nan, nan,  nan,  nan, False)
 (0, nan, nan, nan,  nan,  nan, False)
 (5, nan, nan, nan, 1.4 ,  1.8, False)
 (5, nan, nan, nan, 0.3 ,  1.2, False)
 (0, nan, nan, nan,  nan,  nan, False)
 (5, nan, nan, nan, 0.04,  0.2, False)
 (5, nan, nan, nan, 4.  , 10. , False)
 (5, nan, nan, nan, 0.1 ,  0.3, False)
 (0, nan, nan, nan,  nan,  nan, False)
 (0, nan, nan, nan,  nan,  nan, False)]
[(5, nan, nan, nan, 0.5, 1.7, False) (5, nan, nan, nan, 0.5, 1.2, False)
 (5, nan, nan, nan, 1. , 2.7, False) (5, nan, nan, nan, 1. , 1.7, False)
 (5, nan, nan, nan, 1. , 1.5, False)]


0% [#] 100% | ETA: 00:00:01
Total time elapsed: 00:00:00
/Users/romain/anaconda3/envs/pathways/lib/python3.11/site-packages/scikits/umfpack/umfpack.py:578: UmfpackWarning: Singular matrix
  warnings.warn('Singular matrix', UmfpackWarning)
/Users/romain/anaconda3/envs/pathways/lib/python3.11/site-packages/scikits/umfpack/umfpack.py:727: UmfpackWarning: Zeroing nan and inf entries...
  warnings.warn('Zeroing nan and inf entries...', UmfpackWarning)
/Users/romain/anaconda3/envs/pathways/lib/python3.11/site-packages/scikits/umfpack/umfpack.py:732: RuntimeWarning: divide by zero encountered in scalar divide
  econd = 1.0 / self.info[UMFPACK_RCOND]
/Users/romain/anaconda3/envs/pathways/lib/python3.11/site-packages/scikits/umfpack/umfpack.py:736: UmfpackWarning: (almost) singular matrix! (estimated cond. number: inf)
  warnings.warn(msg, UmfpackWarning)


------ Calculating LCA results for 2050...
[(0, nan, nan, nan,  nan, nan, False) (5, nan, nan, nan, 0.3 , 0.6, False)
 (5, nan, nan, nan, 1.  , 1.4, False) (0, nan, nan, nan,  nan, nan, False)
 (0, nan, nan, nan,  nan, nan, False) (5, nan, nan, nan, 1.4 , 1.8, False)
 (5, nan, nan, nan, 0.2 , 0.7, False) (0, nan, nan, nan,  nan, nan, False)
 (5, nan, nan, nan, 0.04, 0.2, False) (5, nan, nan, nan, 4.  , 8. , False)
 (5, nan, nan, nan, 0.1 , 0.3, False) (0, nan, nan, nan,  nan, nan, False)
 (0, nan, nan, nan,  nan, nan, False)]
[(5, nan, nan, nan, 0.5, 1.7, False) (5, nan, nan, nan, 0.5, 0.9, False)
 (5, nan, nan, nan, 1. , 2.7, False) (5, nan, nan, nan, 1. , 1.7, False)
 (5, nan, nan, nan, 0.8, 1.3, False)]


0% [#] 100% | ETA: 00:00:01
Total time elapsed: 00:00:00
/Users/romain/anaconda3/envs/pathways/lib/python3.11/site-packages/scikits/umfpack/umfpack.py:578: UmfpackWarning: Singular matrix
  warnings.warn('Singular matrix', UmfpackWarning)
/Users/romain/anaconda3/envs/pathways/lib/python3.11/site-packages/scikits/umfpack/umfpack.py:727: UmfpackWarning: Zeroing nan and inf entries...
  warnings.warn('Zeroing nan and inf entries...', UmfpackWarning)
/Users/romain/anaconda3/envs/pathways/lib/python3.11/site-packages/scikits/umfpack/umfpack.py:732: RuntimeWarning: divide by zero encountered in scalar divide
  econd = 1.0 / self.info[UMFPACK_RCOND]
/Users/romain/anaconda3/envs/pathways/lib/python3.11/site-packages/scikits/umfpack/umfpack.py:736: UmfpackWarning: (almost) singular matrix! (estimated cond. number: inf)
  warnings.warn(msg, UmfpackWarning)
0% [####] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


In [3]:
p.lca_results.sum()

<xarray.DataArray ()> Size: 8B
array(12977.17768715)

In [4]:
arr = p.lca_results.interp(
    year=range(p.lca_results.coords["year"].values.min(), p.lca_results.coords["year"].values.max() + 1)
)

In [5]:
df = arr.to_dataframe("value")

In [6]:
df = df[df["value"]!=0.0]
df = df[~df["value"].isnull()]
print(len(df))
df=df.reset_index()

151


In [7]:
from pivottablejs import pivot_ui
from IPython.display import HTML
pivot_ui(df, outfile_path='example.html')